### Import all required libraries and set constants for server connection

In [ ]:
import os, sys
import numpy as np
import pickle
import time
import matplotlib.pyplot as plt
while os.getcwd()[-6:] != 'inkube':
    os.chdir('..')
print(os.getcwd())
sys.path.insert(0, './Communication')
from ControlPort import ControlPort
from Inkuflow import Inkuflow

In [ ]:
# Create instance of ComtrolPort class
ctrl = ControlPort(init_inkulevel=True, MAX_STEPS=72_000, do_log=True)

Control MEA temperature

In [ ]:
ctrl.set_mea_temp(37, [0]) # set ought temperature for MEA A
ctrl.set_mea_control(1, [0]) # turn controller for MEA A on

ctrl.send_commands()

Control reservoir temperature and hunmidity

In [ ]:

ctrl.set_env_control(temperature=1, aux=1, aux_is_humidity=0)
ctrl.set_env_temp(37) 
ctrl.set_humidity(60)

ctrl.send_commands()

Control CO2

In [ ]:
ctrl.set_co2(5)
ctrl.set_co2_control(state=1, calib=1, init=0)

ctrl.send_commands()

Initialise inkuflow

In [ ]:
ctrl.enable_pump()

# this intitialises the class to keep level constant with pump and level instance
mea_valves = np.array([
    [ 0, 1, 2, 3],
    [ 6, 7, 8, 9],
    [12,13,14,15], 
    [18,19,20,21], 
])
reservoir_valves = np.array([
    [4],      
    [10],
    [16],
    [22],  
])
flw = Inkuflow(
    ctrl, 
    reservoir_valves=reservoir_valves, 
    mea_valves=mea_valves, 
    pump_amount_for_const=int(0.02*Inkuflow.PUMP_STEPS_FOR_100uL), 
    active_mea=np.array([1,1,1,1]), 
    medium_lvl_lim_min = np.array([ 2_000, 2_000, 2_000, 2_000]),
    medium_lvl_lim_max = np.array([ 3_000, 3_000, 3_000, 3_000]),
    pump_range_lim=[2_000, 74_000],
)

liquids = {'medium': 0, 'mQ': 1, 'waste': 2, 'EtOH': 3}

const_liquid = 1

Bring pump to midrange position

In [ ]:

ctrl.enable_pump()
flw.switch_reservoir(1)
ctrl.pump_abs(2_000,10)
time.sleep(.5)
ctrl.pump_abs(35_000,10)
ctrl.pump_position = 35_000
ctrl.send_commands()


In [ ]:
flw.switch_reservoir(0) # close valves to reservoir

In [ ]:
# manually switch a valve
state = 1
offset = 0
ctrl.switch_valve([0+offset, 6+offset, 12+offset], [state]*3, 0) 
ctrl.send_commands()

Pump without feedback

In [ ]:
flw.exchange_amount_no_fb(
    exchange_amount_steps=flw.pump_steps_for_100uL*.3,
    liquids=[0], 
    liquid_directions=[-1], 
    target_mea=[0,0,1,0])

Initialise inkulevel

In [ ]:

for _ in range(2):
    ctrl.start_init_inkulevel()
    ctrl.send_commands()
    time.sleep(2)

In [ ]:
rand_add = 0
delay_time = .5
for target in [1,0,2,3]:
    # for upper/lower put the 600-target output by inkulevel divided by 10 as result is inverted
    ctrl.set_inkulevel_param(target_inkulevel=[target] ,param_key='lower', value=[0+rand_add])
    ctrl.send_commands()
    time.sleep(delay_time)
    ctrl.set_inkulevel_param(target_inkulevel=[target] ,param_key='upper', value=[600+rand_add]) 
    ctrl.send_commands()
    time.sleep(delay_time)
    ctrl.set_inkulevel_param(target_inkulevel=[target] ,param_key='thresh', value=[255*1+rand_add])
    ctrl.send_commands()
    time.sleep(delay_time)
    ctrl.set_inkulevel_param(target_inkulevel=[target] ,param_key='exposure', value=[800+rand_add]) 
    ctrl.send_commands()
    time.sleep(delay_time)
    ctrl.set_inkulevel_param(target_inkulevel=[target] ,param_key='minpix', value=[220+rand_add])
    ctrl.send_commands()
    time.sleep(delay_time)
    ctrl.set_inkulevel_param(target_inkulevel=[target] ,param_key='mindist', value=[600+rand_add])
    ctrl.send_commands()
    time.sleep(delay_time)
    ctrl.set_inkulevel_param(target_inkulevel=[target] ,param_key='peaknum', value=[0+rand_add])
    ctrl.send_commands()
    time.sleep(delay_time)
    ctrl.send_commands()

In [ ]:
ctrl.start_inkulevel_debug_bt([0,0,0,2]) # send every 2nd image of inkulevel on MEA A
ctrl.send_commands()

In [ ]:
ctrl.start_inkulevel_send([0,0,0,1]) # activate processing and sending through UART
ctrl.send_commands()

Calibration cycle for feedbacked pumping

In [ ]:
measurements_max = []
measurements_min = []
sleep_time = 20
calibration_iterations = 2
measurement_iterations = 3
calibration_liquid = 0
target = [0,0,1,0]
exchange_amount = flw.pump_steps_for_100uL*.2

# rougly 5 min per cycle (for 4 MEAs)
time.sleep(30)
for i in range(calibration_iterations*2):

    if not i%2: 
        flw.exchange_amount_no_fb(            
            exchange_amount_steps=exchange_amount,
            liquids=[2], 
            liquid_directions=[1], 
            target_mea=target
        )

        print(f'removed for iteration {i}')
        time.sleep(sleep_time)
        measurements_min.append(flw.measure_lvl(measurement_iterations, key='median'))
        print(measurements_min[-1])
    else:
        flw.exchange_amount_no_fb(            
            exchange_amount_steps=exchange_amount,
            liquids=[calibration_liquid], 
            liquid_directions=[-1], 
            target_mea=target
        )

        print(f'added for iteration {i}')
        time.sleep(sleep_time)
        measurements_max.append(flw.measure_lvl(measurement_iterations, key='median'))
        print(measurements_max[-1])



print(measurements_min)
print(measurements_max)
flw.medium_lvl_lim_max = np.mean(np.array(measurements_max), axis=0)
flw.medium_lvl_lim_min = np.mean(np.array(measurements_min), axis=0)

print(f'Max Threshold: {flw.medium_lvl_lim_max}')
print(f'Min Threshold: {flw.medium_lvl_lim_min}')


In [ ]:
# measure current level
a = flw.measure_lvl(3, key='median')

keep volume in wells constant

In [ ]:
# keep level constant
flw.const_level(
    duration_in_s=60*60*1, # 1 hour
    iterations=3, 
    liquid=0, 
    target_lvl=flw.medium_lvl_lim_max, 
)
print('Start constant level')

execute stepwise medium exchange

In [ ]:
# exchange cycle new
flw.const_level(
    duration_in_s=(60*4)*60, 
    iterations=3, 
    liquid=1, 
    target_lvl=flw.medium_lvl_lim_max, 
)
print('Start constant level')

start_t = time.time()
counter = 0
mea_id = 0
active_meas = [2]

exchange_amount_per_hour = 750/4/24 # in ul
minutes_for_20ul = 20/exchange_amount_per_hour * 60
cycle_duration = minutes_for_20ul/len(active_meas)
print(f'Cycle duration in minutes {cycle_duration}')
wait_time_for_exchange = 10

calib_step_size = 0.5*flw.pump_steps_for_100uL
dist_vec = flw.medium_lvl_lim_max - flw.medium_lvl_lim_min
print(f'Distance between thresholds: {dist_vec}')

while counter < (len(active_meas)*24*5):
    target_mea = active_meas[mea_id]
    print(f'Start cycle {target_mea}')

    flw.pump_until_single_open(
        measurement_iterations=2,
        liquid=2,
        target_lvl=flw.medium_lvl_lim_min[target_mea],
        target_mea=target_mea,
        direction=1,
        anticipated_step_size=calib_step_size,
        anticipated_dist = dist_vec[target_mea],
        max_factor=1.5,
        init_measurement=None,
        wait_time_before_measure=10,
        approach_factor = .6,
        max_iterations = 4, 
        discard_measurements=2, 
        min_step_size = 300, 
        adapt_step_size = False, 
    )
    time.sleep(10)
    flw.pump_until_single_open(
        measurement_iterations=2,
        liquid=0,
        target_lvl=flw.medium_lvl_lim_max[target_mea],
        target_mea=target_mea,
        direction=-1,
        anticipated_step_size=calib_step_size,
        anticipated_dist = dist_vec[target_mea],
        max_factor=2,
        init_measurement=None,
        wait_time_before_measure=10,
        approach_factor = .6,
        max_iterations = 4, 
        discard_measurements=2, 
        min_step_size = 300, 
        adapt_step_size = False, 
    )
    
    time.sleep(10)

    # wait here until n*hours + 10 min
    while ((time.time() - (start_t+counter*60*cycle_duration)) < wait_time_for_exchange*60):
        time.sleep(5)
    print('Start constant level')
    flw.const_level(
        duration_in_s=(cycle_duration-wait_time_for_exchange)*60, 
        iterations=3, 
        liquid=1, 
        target_lvl=flw.medium_lvl_lim_max, 
    )
    print(f'Finished cycle {counter} after {(time.time()-start_t)/60} min')
    counter += 1
    mea_id = (mea_id+1) % len(active_meas)
